In [81]:
import pandas as pd
import numpy as np
import plotly.express as px

In [82]:
store_data_df = pd.read_csv('../data/raw/dmml1_stores.csv')
test_data_df = pd.read_csv('../data/raw/dmml1_test.csv')
train_data_df = pd.read_csv('../data/raw/dmml1_train.csv')

In [83]:
store_data_df

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1,2,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
3,4,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
4,5,a,c,4590.0,3.0,2000.0,1,40.0,2011.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...
295,296,a,a,690.0,6.0,2007.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
296,297,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN
297,298,d,c,1340.0,10.0,2006.0,1,5.0,2013.0,"Feb,May,Aug,Nov"
298,299,d,a,260.0,2.0,2012.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [84]:
train_data_df

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,4,2015-04-30,6884,716,1,1,0,0
1,1,3,2015-04-29,6764,756,1,1,0,0
2,1,2,2015-04-28,6861,678,1,1,0,0
3,1,1,2015-04-27,6523,647,1,1,0,0
4,1,7,2015-04-26,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
246898,300,6,2013-01-05,5194,569,1,0,0,0
246899,300,5,2013-01-04,5524,646,1,0,0,1
246900,300,4,2013-01-03,5563,718,1,0,0,1
246901,300,3,2013-01-02,6218,753,1,0,0,1


In [49]:
# take dftrain and join with dfstores
dfmerge = train_data_df.merge(store_data_df[['Store ID', 'StoreType', 'Assortment']], on='Store ID', how='left')

In [50]:
dfmerge

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment
0,1,4,2015-04-30,6884,716,1,1,0,0,a,a
1,1,3,2015-04-29,6764,756,1,1,0,0,a,a
2,1,2,2015-04-28,6861,678,1,1,0,0,a,a
3,1,1,2015-04-27,6523,647,1,1,0,0,a,a
4,1,7,2015-04-26,0,0,0,0,0,0,a,a
...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,2013-01-05,5194,569,1,0,0,0,a,c
246899,300,5,2013-01-04,5524,646,1,0,0,1,a,c
246900,300,4,2013-01-03,5563,718,1,0,0,1,a,c
246901,300,3,2013-01-02,6218,753,1,0,0,1,a,c


# 1. Wie viele Datenpunkte gibt es pro Store?

In [51]:
store_counts = train_data_df.groupby('Store ID').size().reset_index()
store_counts.columns = ['Store ID', 'Data Points']
store_counts

,Store ID,Data Points
0,1,850
1,2,850
2,3,850
3,4,850
4,5,850
...,...,...
295,296,850
296,297,850
297,298,850
298,299,666


In [53]:
unique_values = store_counts['Data Points'].value_counts().sort_index()

In [80]:
for i, v in unique_values.items():
    print(f"Anzahl Datenpunkte: {i} + Stores: {v}")

print(f"Die Anzahl der eindeutigen Werte ist: {len(unique_values)}")


Anzahl Datenpunkte: 666 - Stores: 44
Anzahl Datenpunkte: 849 - Stores: 1
Anzahl Datenpunkte: 850 - Stores: 255
Die Anzahl der eindeutigen Werte ist: 3


# Visualisieren Sie die Verteilung der Verkaufszahlen für einige zufällig ausgewählte Stores und be- schreiben Sie typische Muster, die Sie erkennen.

In [ ]:
# Wählen Sie zufällig 5 Stores aus
random_stores = np.random.choice(dftrain['Store ID'].unique(), 5)

# Erstellen Sie ein Plot für jeden Store
for store in random_stores:
    store_data = train_data_df[train_data_df['Store ID'] == store]
    fig = px.line(store_data, x='Date', y='Sales', title=f'Sales for Store {store}')
    fig.show()

In [97]:
store_data_df

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1,2,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
3,4,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
4,5,a,c,4590.0,3.0,2000.0,1,40.0,2011.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...
295,296,a,a,690.0,6.0,2007.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
296,297,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN
297,298,d,c,1340.0,10.0,2006.0,1,5.0,2013.0,"Feb,May,Aug,Nov"
298,299,d,a,260.0,2.0,2012.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [98]:
# all data where store_data_df.Promo2SinceYear is nan
missingnan = store_data_df[store_data_df['Promo2'] == 0 ]

In [99]:
missingnan

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
2,3,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
5,6,d,c,9800.0,7.0,2012.0,0,NaN,NaN,NaN
6,7,c,a,2240.0,9.0,2009.0,0,NaN,NaN,NaN
8,9,d,a,1090.0,4.0,2007.0,0,NaN,NaN,NaN
10,11,d,a,9710.0,2.0,2014.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
285,286,d,c,12020.0,7.0,2009.0,0,NaN,NaN,NaN
290,291,a,c,1410.0,10.0,2013.0,0,NaN,NaN,NaN
294,295,c,c,10450.0,6.0,2009.0,0,NaN,NaN,NaN
296,297,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN


In [73]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer


# Bereinigung der Daten
# Entfernen von Zeilen, in denen der Laden geschlossen ist, da sie keinen Umsatz generieren.
train_data_cleaned = train_data_df[train_data_df['Open'] == 1].drop(columns=['Open'])

# Zusammenführen der Trainings- und Store-Daten
merged_data = pd.merge(train_data_cleaned, store_data_df, on='Store ID', how='left')

# Umwandlung des Datums in ein Datumstyp und Extraktion von Jahr und Monat
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data['Year'] = merged_data['Date'].dt.year
merged_data['Month'] = merged_data['Date'].dt.month
merged_data.drop(columns=['Date'], inplace=True)  # Datumsspalte entfernen

# Identifizierung der numerischen und kategorialen Spalten
numerical_columns = merged_data.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = merged_data.select_dtypes(include=['object']).columns

# Behandlung von fehlenden Werten
# Numerische Spalten: Ersetzen durch Median
numerical_imputer = SimpleImputer(strategy='median')
merged_data[numerical_columns] = numerical_imputer.fit_transform(merged_data[numerical_columns])

# Kategoriale Spalten: Ersetzen durch Modus
categorical_imputer = SimpleImputer(strategy='most_frequent')
merged_data[categorical_columns] = categorical_imputer.fit_transform(merged_data[categorical_columns])

# One-Hot-Encoding für kategoriale Variablen
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_categorical_data = encoder.fit_transform(merged_data[categorical_columns])
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_columns))

# Entfernen der ursprünglichen kategorialen Spalten und Hinzufügen der kodierten Spalten
merged_data_encoded = merged_data.drop(columns=categorical_columns).join(encoded_categorical_df)

# Skalierung der numerischen Features
scaler = StandardScaler()
scaled_numerical_data = scaler.fit_transform(merged_data_encoded[numerical_columns])
scaled_numerical_df = pd.DataFrame(scaled_numerical_data, columns=numerical_columns)

# Zusammenführen der skalierten numerischen und kodierten kategorialen Daten
final_preprocessed_data = scaled_numerical_df.join(encoded_categorical_df)


/Users/wladi/opt/anaconda3/envs/bv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.

